# Eksempler kode for grunnleggene kurs i R 

### R som kalkulator

In [ ]:
2 + 2

In [ ]:
25 * 4


In [ ]:
# Et lite tall
3E-3

In [ ]:
# Et stort tall
3E3

In [ ]:
# log av et tall (natural)
log(10)

### R som tekststreng

In [ ]:
"Hello, world!"

In [ ]:
# Lim to tekster sammen til en tekststreng
paste("Hallo", "Aslaug")

In [ ]:
# lime to tekser sammen til en tekststreng med komma mellom
paste("Hallo", "Aslaug", sep = ", ")

In [ ]:
# lim tekst og tall sammen til en tekststreng
paste("Aslaug er født i ", 1971, " i Levanger")

In [ ]:
# Plukk ut noen character i en string
substr("Susie liker kake", 1, 5)


### R som objekter

In [ ]:

grunnlagt <- 1669
grunnlagt


In [ ]:

sted <- "Kongsvinger Festning"
sted


In [ ]:
# Objekter kan overskrives
grunnlagt <- grunnlagt + 1
grunnlagt

sted <- "Oslo"
sted

In [ ]:
# Objekttype
str(sted)
str(grunnlagt)


### R som liste eller vektor

In [ ]:

# Eksempel på en numerisk vector 
aar <- c(2018, 2019, 2020)
aar


In [ ]:

# Eksempel på en kategoriske vektor
flere_sted <- c("Oslo","Kongsvinger", "Rogaland")
flere_sted

# Hvor mange er i vektoren
length(flere_sted)


### Logiske setninger

In [ ]:
grunnlagt == 1669
grunnlagt == 1670
aar == 2018

In [ ]:
#test for motsetninger
2 != 3

In [ ]:
# test for innhold
2018 %in% aar
2015 %in% aar

ifelse(sted == "Oslo", "0301", "3401")

### Lage datasett

In [ ]:
steddata <- data.frame(flere_sted, aar)
steddata

In [ ]:

fylkedata <- data.frame(fylke = c("Viken",  "Innlandet"), innbyggere = c(1241000, 371000))
View(fylkedata)

### Lese inn data

In [ ]:
#### Les inn csv fil ####
library(tidyverse)

# Datasett med komma skilletegn
kommunedata <- read_csv("./data/kommunedata.csv")

kommunedata

In [ ]:
# Datasett med ; skilletegn
sykemeldingdata <- read_csv2("./data/sykemelding.csv")
View(sykemeldingdata)

In [ ]:
#### Les inn SAS-fil ####
library(haven)
trygd <- read_sas("./data/trygd.sas7bdat")
View(trygd)

?read_sas

### Ser næremere på data

In [ ]:
glimpse(kommunedata)

In [ ]:
summary(kommunedata)

In [ ]:
names(kommunedata)

## Databearbeiding i R tidyverse

### Ny variabel - mutate

In [ ]:
# Lage en ny variabel og log bruttointekt
kommunedata <- kommunedata %>%
  mutate(log_bruttoinnt = log(bruttoinnt + 1))
glimpse(kommunedata)


In [ ]:
# Lage en variabel med ifelse
kommunedata <- kommunedata %>%
  mutate(region_type = ifelse(nchar(region) == 4, "kommune", "andre"))


In [ ]:
# Endre variabeltype for tid
kommunedata <- kommunedata %>%
  mutate(tid = as.character(tid),
         kjonn = as.character(kjonn))

### Selektere ut rader

In [ ]:
# ser på data fra hele Norge
kommunedata %>%
  filter(region == "0")

In [ ]:
# Ser på data fra årene 2016 og 2017
kommunedata %>%
  filter(tid %in% c("2016", "2017"))

In [ ]:
# Ser på data fra hele Norge for 2018
kommunedata %>%
  filter(region == "0", tid == "2017")


### Velge noen variabler

In [ ]:
#### Velg ut noen variabler ####
kommunedata %>%
  select(region, kjonn, tid, folkemengde)


kommunedata %>%
  filter(region == "0", kjonn == 2) %>%
  select(tid, bruttoinnt)


### Aggregering - gjennomsnitt - antall

In [ ]:
kommunedata %>%
  summarise(average = mean(folkemengde))

In [ ]:
kommunedata %>%
  filter(region_type == "kommune", kjonn == 1) %>%
  summarise(ave_bruttoinnt = mean(bruttoinnt), antall = n())

# NB: dette er gjennomsnitt og antall for alle årene!

### Gruppering

In [ ]:
#### Gruppering ####
kommunedata %>%
  filter(region_type == "kommune", kjonn == 1) %>%
  group_by(tid) %>%
  summarise(ave_bruttoinnt = mean(bruttoinnt), antall = n())

In [ ]:
# antall kommune etter år og kjønn
kommunedata %>%
  filter(region_type == "kommune") %>%
  group_by(tid, kjonn) %>%
  summarise(antall = n()) 

In [ ]:

kommunedata %>%
  filter(region_type == "kommune") %>%
  group_by(tid, kjonn) %>%
  summarise(antall = n()) %>%
  spread(tid, antall)


In [ ]:
#### Endre variabelnavn ####
kommunedata %>%
  rename(aar = tid)


### Koble datasett

In [ ]:
library(haven)
trygd <- read_sas("./data/trygd.sas7bdat")


# Endrenavn i trygd datasett
names(trygd)
trygd <- trygd %>%
  rename(region = Region,
         kjonn = Kjonn,
         tid = Tid
  )
glimpse(trygd)


In [ ]:
# koble
kommunedata <- left_join(kommunedata, trygd, by = c("region", "tid", "kjonn")) 
glimpse(kommunedata)

### Grafikk med ggplot

In [ ]:
#### Søylediagram ####
kommunedata %>% ggplot(aes(kjonn)) + 
  geom_bar()

In [ ]:
# Antall uforetrygd etter kjønn
kommunedata %>%
  ggplot(aes(x = kjonn, y = UforetygdPers)) +
  geom_bar(stat = "identity")

In [ ]:
# Endre format på y-aksen
kommunedata %>%
  ggplot(aes(x = kjonn, y = UforetygdPers)) +
  geom_bar(stat = "identity") +
  scale_y_continuous(labels = scales::comma)


In [ ]:
# Vise forskjellige år
kommunedata %>%
  ggplot(aes(x = kjonn, y = UforetygdPers, fill = tid)) +
  geom_bar(stat = "identity") +
  scale_y_continuous(labels = scales::comma) 

In [ ]:
# Vise forskjellige år ved siden
kommunedata %>%
  ggplot(aes(x = kjonn, y = UforetygdPers, fill = tid)) +
  geom_bar(stat = "identity", position = "dodge") +
  scale_y_continuous(labels = scales::comma) 

In [ ]:
# Filter for Kongsvinger
kommunedata %>%
  filter(region == "0402") %>% 
  ggplot(aes(x = kjonn, y = UforetygdPers, fill = tid)) +
  geom_bar(stat = "identity", position = "dodge") + 
  scale_y_continuous(labels = scales::comma) 


In [ ]:
#### Punktdiagram ####
kommunedata %>%
  ggplot(aes(x = folkemengde, y = bruttoinnt)) + 
  geom_point() + 
  scale_y_continuous(labels = scales::comma) +
  scale_x_continuous(labels = scales::comma) 

In [ ]:
# Filter for kun kommune-nivå data
kommunedata %>%
  filter(region_type == "kommune") %>%
  ggplot(aes(x = folkemengde, y = bruttoinnt)) + 
  geom_point() + 
  scale_y_continuous(labels = scales::comma) +
  scale_x_continuous(labels = scales::comma) 

In [ ]:
# Legg til regresjonslinje med geom_smotth
kommunedata %>%
  filter(region_type == "kommune") %>%
  ggplot(aes(x = folkemengde, y = bruttoinnt)) + 
  geom_point() + 
  scale_y_continuous(labels = scales::comma) +
  scale_x_continuous(labels = scales::comma) +
  geom_smooth(method = "lm")

In [ ]:
# Farge etter kjønn
kommunedata %>%
  filter(region_type == "kommune") %>% 
  ggplot(aes(x = folkemengde, y = bruttoinnt)) + 
  geom_point(aes(color = kjonn)) +
  scale_y_continuous(labels = scales::comma) +
  scale_x_continuous(labels = scales::comma)


In [ ]:
# Gir akser et nytt navn
fig <-kommunedata %>%
  filter(region_type == "kommune") %>% 
  ggplot(aes(x = folkemengde, y = bruttoinnt)) + 
  geom_point(aes(color = kjonn)) +
  scale_y_continuous(labels = scales::comma, name = "Brutto inntekt") +
  scale_x_continuous(labels = scales::comma, name = "Antall personer")
fig

### Interaktive figurer - Plotly

In [ ]:
library(plotly)
ggplotly(fig)
